In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

import numpy as np
from numpy import linalg
import math


In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


Default GPU Device: /device:GPU:0


In [ ]:
#load dataset
# do not use zero
df= pd.read_csv('./ML-CUP23-TR.csv', sep=",", low_memory=False, header = None,  usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13], names=[1,2,3,4,5,6,7,8,9,10,'x','y','z'], skiprows=range(0, 7))
y_labels=df[['x','y','z']]
features_df = df[[1,2,3,4,5,6,7,8,9,10]]
display(df)

,1,2,3,4,5,6,7,8,9,10,x,y,z
0,1,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382
1,2,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571
2,3,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090
3,4,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777
4,5,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,-0.803560,-0.878859,-0.978391,0.957539,0.996655,0.996924,-0.226305,0.097814,-0.922666,0.934153,15.389553,-41.068806
996,997,0.718841,0.995748,-0.942678,0.999929,0.747763,-0.808726,0.176132,0.999100,-0.753970,-0.969009,-36.228770,13.067430
997,998,-0.977912,-0.971108,0.956233,-0.979524,0.222033,0.986609,0.658273,-0.987310,0.937697,0.143420,7.265506,-53.497242
998,999,0.284803,-0.988684,-0.427197,0.883317,0.993302,0.999500,-0.019456,-0.648110,-0.955231,0.901298,5.545274,-63.348396


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features_df, y_labels, stratify =None, test_size=0.40, random_state = 42, shuffle=False)

# Functions Definition

In [ ]:
tf.keras.utils.set_random_seed(42)

In [ ]:
def perform_complete_grid_search(train_x, train_y, param_grid):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    all_models = []

    combination_count = 0  # Counter for combinations

    total_combinations = len(param_grid['units1']) * len(param_grid['activation']) * len(param_grid['l2_reg']) * len(param_grid['units2']) * len(param_grid['units3']) * len(param_grid['dropout_rate']) * len(param_grid['learning_rate'])
    print(f"Total Combinations: {total_combinations}")

    for units1 in param_grid['units1']:
        for activation in param_grid['activation']:
            for l2_reg in param_grid['l2_reg']:
                for units2 in param_grid['units2']:
                    for units3 in param_grid['units3']:
                        for dropout_rate in param_grid['dropout_rate']:
                            for learning_rate in param_grid['learning_rate']:
                                fold_scores = []
                                for train_index, val_index in kf.split(train_x):
                                    X_train_fold, X_val_fold = train_x.iloc[train_index], train_x.iloc[val_index]
                                    y_train_fold, y_val_fold = train_y.iloc[train_index], train_y.iloc[val_index]

                                    scaler = StandardScaler()
                                    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
                                    X_val_fold_scaled = scaler.transform(X_val_fold)
                                    tf.keras.backend.clear_session() # releases the global state: this helps
                                                                      # avoid clutter from old models and layers
                                    tf.keras.utils.set_random_seed(42)
                                    model = create_model(units1=units1, activation1=activation, l2_reg1=l2_reg,
                                                         units2=units2, activation2=activation, l2_reg2=l2_reg,
                                                         units3=units3, activation3=activation, l2_reg3=l2_reg,
                                                         dropout_rate=dropout_rate, learning_rate=learning_rate)
                                    with tf.device("device:GPU:0"):
                                      model.fit(X_train_fold_scaled, y_train_fold, epochs=100, batch_size=64, verbose=0)

                                    val_pred = model.predict(X_val_fold_scaled)
                                    val_mse = mean_squared_error(y_val_fold, val_pred)
                                    fold_scores.append(val_mse)

                                avg_mse = np.mean(fold_scores)
                                std_mse = np.std(fold_scores)

                                all_models.append({
                                    'params': {
                                        'units1': units1, 'activation1': activation, 'l2_reg': l2_reg,
                                        'units2': units2, 'activation2': activation, 'l2_reg': l2_reg,
                                        'units3': units3, 'activation3': activation, 'l2_reg': l2_reg,
                                        'dropout_rate': dropout_rate, 'learning_rate': learning_rate
                                    },
                                    'avg_mse': avg_mse,
                                    'std_mse': std_mse
                                })

                                combination_count += 1
                                print(f"Combination {combination_count}:")
                                print(f"   Units1: {units1}, Activation1: {activation}, L2_reg: {l2_reg}")
                                print(f"   Units2: {units2}, Activation2: {activation}, L2_reg: {l2_reg}")
                                print(f"   Units3: {units3}, Activation3: {activation}, L2_reg: {l2_reg}")
                                print(f"   Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}")
                                print(f"   Average MSE: {avg_mse:.4f}, Std MSE: {std_mse:.4f}")

    # Sort all models by the average MSE
    all_models.sort(key=lambda x: x['avg_mse'])

    return all_models


In [ ]:
def create_model_base(units1=32, activationf='tanh', initializer1='random_uniform', l2_reg=0.01,
                 units2=32, initializer2='random_uniform',
                 units3=32, initializer3='random_uniform',
                 dropout_rate=0.1, learning_rate=0.01, beta_1=0.9, beta_2=0.999):
    model = Sequential()
    model.add(Dense(units1, activation=activationf, kernel_initializer=initializer1,
                    kernel_regularizer=l2(l2_reg), input_shape=(10,)))  # Adjust the input_shape based on your data
    model.add(Dropout(dropout_rate))
    model.add(Dense(units2, activation=activationf, kernel_initializer=initializer2,
                    kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units3, activation=activationf, kernel_initializer=initializer3,
                    kernel_regularizer=l2(l2_reg)))
    model.add(Dense(3, activation='linear'))  # Adjust the output layer based on your requirements
    model.compile(optimizer=Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2), loss='mean_squared_error')
    return model


Model create, default parameters are the one resulting from the hyperparameters tuning

In [ ]:
def create_model(units1=32, activation1='tanh', initializer1='glorot_uniform', l2_reg1=0.1,
                 units2=32, activation2='tanh', initializer2='random_uniform', l2_reg2=3.1,
                 units3=32, activation3='relu', initializer3='random_normal', l2_reg3=0.1,
                 dropout_rate=0.1, learning_rate=0.01, beta_1=0.99, beta_2=0.999):
    model = Sequential()
    model.add(Dense(units1, activation=activation1, kernel_initializer=initializer1,
                    kernel_regularizer=l2(l2_reg1), input_shape=(10,)))  # Adjust the input_shape based on your data
    model.add(Dropout(dropout_rate))
    model.add(Dense(units2, activation=activation2, kernel_initializer=initializer2,
                    kernel_regularizer=l2(l2_reg2)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units3, activation=activation3, kernel_initializer=initializer3,
                    kernel_regularizer=l2(l2_reg3)))
    model.add(Dense(3, activation='linear'))  # Adjust the output layer based on your requirements
    model.compile(optimizer=Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2), loss='mean_squared_error')
    return model

(Exhaustive)GridSearch cross validation definition

In [ ]:
def GridSearchCV_Call(param_grid):
  keras_model = KerasRegressor(model=create_model_base, epochs=100, batch_size=64, verbose=0)

  # Create a Pipeline
  pipeline = Pipeline([
      ('scaler', StandardScaler()),
      ('model', keras_model)
  ])

  #pipeline.set_params(model__c= param_dist)
  # Initialize RandomizedSearchCV
  grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid,
                                     cv=5, verbose=19,scoring='neg_mean_squared_error',n_jobs=-1)

  # Perform the search (assuming train_x and train_y are already defined)
  grid_search.fit(train_x, train_y)
  return grid_search

RandomGridSearch cross validation definition

In [ ]:
def RandomizedSearchCV_Call(param_dist,n_iter=60):
  keras_model = KerasRegressor(model=create_model, epochs=100, batch_size=64, verbose=0)

  # Create a Pipeline
  pipeline = Pipeline([
      ('scaler', StandardScaler()),
      ('model', keras_model)
  ])

  #pipeline.set_params(model__c= param_dist)
  # Initialize RandomizedSearchCV
  random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist,
                                    n_iter=n_iter , cv=5, verbose=19, random_state=42,scoring='neg_mean_squared_error')

  # Perform the search (assuming train_x and train_y are already defined)
  random_search.fit(train_x, train_y)
  return random_search

Return Top 10 from RandomSearchCV

In [ ]:
def listing_top_10(search):
  # Retrieve the top 10 models
  top_10_indices = np.argsort(-search.cv_results_['mean_test_score'])[:10]
  top_10_scores = search.cv_results_['mean_test_score'][top_10_indices]
  top_10_std = search.cv_results_['std_test_score'][top_10_indices]
  top_10_params = [search.cv_results_['params'][i] for i in top_10_indices]

  for score, std, params in zip(top_10_scores, top_10_std, top_10_params):
      print(f"Score: {score}, Std. Dev: {std}, Parameters: {params}")


Plot of K Fold

In [ ]:
def plot_aggregated_histories(all_histories):
    # Determine the minimum length of history among all folds
    min_length = min(len(hist.history['loss']) for hist in all_histories)

    # Truncate all histories to the minimum length
    truncated_metric_values = [hist.history['loss'][:min_length] for hist in all_histories]
    truncated_val_metric_values = [hist.history['val_loss_fold' ][:min_length] for hist in all_histories]

    # Calculate the mean and standard deviation for the truncated histories
    mean_metric_values = np.mean(truncated_metric_values, axis=0)
    mean_val_metric_values = np.mean(truncated_val_metric_values, axis=0)
    std_metric_values = np.std(truncated_metric_values, axis=0)
    std_val_metric_values = np.std(truncated_val_metric_values, axis=0)

    # Plotting
    epochs = range(1, min_length + 1)
    plt.figure(figsize=(12, 6))

    # Plot training metric
    plt.plot(epochs, mean_metric_values, label='Mean Training loss')
    plt.fill_between(epochs, mean_metric_values - std_metric_values, mean_metric_values + std_metric_values, alpha=0.3, color='blue')  # Fill between the lines with blue color

    # Plot validation metric
    plt.plot(epochs, mean_val_metric_values, linestyle='dashdot', label='Mean Validation loss' )
    plt.fill_between(epochs, mean_val_metric_values - std_val_metric_values, mean_val_metric_values + std_val_metric_values, alpha=0.25, color='orange')  # Fill between the lines with orange color

    plt.title('Mean Training and Validation Mean Squared Error (MSE) per Epoch in a 5-Fold Cross-Validation')
    plt.xlabel('Epochs')
    plt.ylabel('MSE')
    plt.legend()
    plt.show()


In [ ]:
def plot_aggregated_histories2(all_histories):
    # Determine the minimum length of history among all folds
    min_length = min(len(hist.history['loss']) for hist in all_histories)

    # Truncate all histories to the minimum length
    truncated_metric_values = [hist.history['loss'][:min_length] for hist in all_histories]
    truncated_val_metric_values = [hist.history['val_loss_fold'][:min_length] for hist in all_histories]

    # Calculate the mean and standard deviation for the truncated histories
    mean_metric_values = np.mean(truncated_metric_values, axis=0)
    mean_val_metric_values = np.mean(truncated_val_metric_values, axis=0)
    std_metric_values = np.std(truncated_metric_values, axis=0)
    std_val_metric_values = np.std(truncated_val_metric_values, axis=0)

    # Plotting
    epochs = range(1, min_length + 1)
    plt.figure(figsize=(12, 6))

    # Plot training metric
    plt.plot(epochs, mean_metric_values, label='Mean Training loss')
    plt.fill_between(epochs, mean_metric_values - std_metric_values, mean_metric_values + std_metric_values, alpha=0.3, color='blue')  # Fill between the lines with blue color

    # Plot validation metric
    plt.plot(epochs, mean_val_metric_values, linestyle='dashdot', label='Mean Validation loss')
    plt.fill_between(epochs, mean_val_metric_values - std_val_metric_values, mean_val_metric_values + std_val_metric_values, alpha=0.25, color='orange')  # Fill between the lines with orange color

    plt.title('Mean Training and Validation Mean Squared Error (MSE) per Epoch in a 5-Fold Cross-Validation')
    plt.xlabel('Epochs')
    plt.ylabel('MSE')
    plt.ylim(0, 100)  # Set the limits of the y-axis
    plt.legend()
    plt.show()


MEE function

In [ ]:
def mean_euclidean_error(y_true, y_pred):
    return np.mean(np.linalg.norm(y_true - y_pred, axis=1))



List top 10

In [ ]:
def print_model_collection(models):
    print("Top Model Configurations:")
    for i, model in enumerate(models, start=1):
        params = model['params']
        avg_mse = model['avg_mse']
        std_mse = model['std_mse']
        print(f"Model {i}:")
        print(f"   Average MSE: {avg_mse:.4f}, Std MSE: {std_mse:.4f}")
        for param, value in params.items():
            print(f"   {param}: {value}")
        print()



Grid Search with batchsize, patience and learning rate reduce

In [ ]:
def perform_callbacks_grid_search(train_x, train_y, param_grid, fixed_param_grid):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    all_models = []

    combination_count = 0  # Initialize combination count

    total_combinations = len(param_grid['batch_size']) * len(param_grid['patience_es']) * len(param_grid['patience_lr']) * len(param_grid['factor_lr'])
    print(f"Total Combinations: {total_combinations}")

    for batch_size in param_grid['batch_size']:
        for patience_es in param_grid['patience_es']:
            for patience_lr in param_grid['patience_lr']:
                for factor_lr in param_grid['factor_lr']:
                    fold_scores = []
                    for train_index, val_index in kf.split(train_x):
                        X_train_fold, X_val_fold = train_x.iloc[train_index], train_x.iloc[val_index]
                        y_train_fold, y_val_fold = train_y.iloc[train_index], train_y.iloc[val_index]

                        scaler = StandardScaler()
                        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
                        X_val_fold_scaled = scaler.transform(X_val_fold)
                        tf.keras.backend.clear_session()
                        tf.keras.utils.set_random_seed(42)

                        model = create_model(units1=fixed_param_grid['units1'], activation1=fixed_param_grid['activation'], l2_reg1=fixed_param_grid['l2_reg'],
                                             units2=fixed_param_grid['units2'], activation2=fixed_param_grid['activation'], l2_reg2=fixed_param_grid['l2_reg'],
                                             units3=fixed_param_grid['units3'], activation3=fixed_param_grid['activation'], l2_reg3=fixed_param_grid['l2_reg'],
                                             dropout_rate=fixed_param_grid['dropout_rate'], learning_rate=fixed_param_grid['learning_rate'])

                        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, verbose=0, restore_best_weights=True)
                        reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, verbose=0)

                        model.fit(X_train_fold_scaled, y_train_fold, epochs=300, batch_size=batch_size, verbose=0, validation_split=0.1, callbacks=[early_stopping, reduce_lr_on_plateau])

                        val_pred = model.predict(X_val_fold_scaled)
                        val_mse = mean_squared_error(y_val_fold, val_pred)
                        fold_scores.append(val_mse)

                    avg_mse = np.mean(fold_scores)
                    std_mse = np.std(fold_scores)

                    combination_count += 1  # Increment combination count
                    print(f"Combination {combination_count}/{total_combinations}: Batch Size={batch_size}, Patience ES={patience_es}, Patience LR={patience_lr}, Factor LR={factor_lr}, Avg MSE={avg_mse:.4f}, Std MSE={std_mse:.4f}")

                    all_models.append({
                        'params': {
                            'batch_size': batch_size,
                            'patience_es': patience_es,
                            'patience_lr': patience_lr,
                            'factor_lr': factor_lr
                        },
                        'avg_mse': avg_mse,
                        'std_mse': std_mse
                    })

    all_models.sort(key=lambda x: x['avg_mse'])
    return all_models


Define callback for validation set history is cross validation

In [ ]:
from tensorflow.keras.callbacks import Callback

class AdditionalValidationSets(Callback):
    def __init__(self, validation_sets, verbose=0):
        super().__init__()
        self.validation_sets = validation_sets
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for val_set_name, val_data in self.validation_sets.items():
            val_X, val_y = val_data
            val_loss = self.model.evaluate(val_X, val_y, verbose=self.verbose)

            logs[f'val_loss_fold'] = val_loss


## Local Execution functions
Functions used in local executions that we could not use in colab as it did not have scikeras
Install scikeras with to use pip install scikeras


In [ ]:
from scikeras.wrappers import KerasRegressor

ModuleNotFoundError: No module named 'scikeras'

In [ ]:
def create_model_base(units1=32, activationf='tanh', initializer1='random_uniform', l2_reg=0.01,
                 units2=32, initializer2='random_uniform',
                 units3=32, initializer3='random_uniform',
                 dropout_rate=0.1, learning_rate=0.01, beta_1=0.9, beta_2=0.999):
    model = Sequential()
    model.add(Dense(units1, activation=activationf, kernel_initializer=initializer1,
                    kernel_regularizer=l2(l2_reg), input_shape=(10,)))  # Adjust the input_shape based on your data
    model.add(Dropout(dropout_rate))
    model.add(Dense(units2, activation=activationf, kernel_initializer=initializer2,
                    kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units3, activation=activationf, kernel_initializer=initializer3,
                    kernel_regularizer=l2(l2_reg)))
    model.add(Dense(3, activation='linear'))  # Adjust the output layer based on your requirements
    model.compile(optimizer=Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2), loss='mean_squared_error')
    return model


(Exhaustive)GridSearch cross validation definition

In [ ]:
def GridSearchCV_Call(param_grid):
  keras_model = KerasRegressor(model=create_model_base, epochs=100, batch_size=64, verbose=0)

  # Create a Pipeline
  pipeline = Pipeline([
      ('scaler', StandardScaler()),
      ('model', keras_model)
  ])

  #pipeline.set_params(model__c= param_dist)
  # Initialize RandomizedSearchCV
  grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid,
                                     cv=5, verbose=19,scoring='neg_mean_squared_error',n_jobs=-1)

  # Perform the search (assuming train_x and train_y are already defined)
  grid_search.fit(train_x, train_y)
  return grid_search

RandomGridSearch cross validation definition

In [ ]:
def RandomizedSearchCV_Call(param_dist,n_iter=60):
  keras_model = KerasRegressor(model=create_model, epochs=100, batch_size=64, verbose=0)

  # Create a Pipeline
  pipeline = Pipeline([
      ('scaler', StandardScaler()),
      ('model', keras_model)
  ])

  #pipeline.set_params(model__c= param_dist)
  # Initialize RandomizedSearchCV
  random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist,
                                    n_iter=n_iter , cv=5, verbose=19, random_state=42,scoring='neg_mean_squared_error')

  # Perform the search (assuming train_x and train_y are already defined)
  random_search.fit(train_x, train_y)
  return random_search

Return Top 10 from RandomSearchCV

In [ ]:
def listing_top_10(search):
  # Retrieve the top 10 models
  top_10_indices = np.argsort(-search.cv_results_['mean_test_score'])[:10]
  top_10_scores = search.cv_results_['mean_test_score'][top_10_indices]
  top_10_std = search.cv_results_['std_test_score'][top_10_indices]
  top_10_params = [search.cv_results_['params'][i] for i in top_10_indices]

  for score, std, params in zip(top_10_scores, top_10_std, top_10_params):
      print(f"Score: {score}, Std. Dev: {std}, Parameters: {params}")


# GridSearchCV1-2

1h 48 min
648 combinations

In [ ]:
param_grid = {
    'units1': [32, 64,128],
    'units2': [32, 64,128],
    'units3': [32, 64,128],
    'activation': ['relu', 'tanh'],
    'l2_reg': [0.01, 0.001],
    'dropout_rate': [0.1,0.2],
    'learning_rate': [0.01,0.002,0.001]
}
perform_complete_grid_search(train_x, train_y, param_grid)




MSE is from average of validation in folds

1. **Avg MSE: 2.1809, Std MSE: 0.4736**
   - Units1: 128, Activation1: relu, L2_reg: 0.01, Units2: 128, Units3: 64, Dropout Rate: 0.1, Learning Rate: 0.01

2. **Avg MSE: 2.2610, Std MSE: 0.6313**
   - Units1: 128, Activation1: relu, L2_reg: 0.001, Units2: 128, Units3: 64, Dropout Rate: 0.1, Learning Rate: 0.01

3. **Avg MSE: 2.3051, Std MSE: 0.3186**
   - Units1: 128, Activation1: relu, L2_reg: 0.001, Units2: 128, Units3: 32, Dropout Rate: 0.1, Learning Rate: 0.01

4. **Avg MSE: 2.3392, Std MSE: 0.3667**
   - Units1: 64, Activation1: relu, L2_reg: 0.001, Units2: 128, Units3: 64, Dropout Rate: 0.1, Learning Rate: 0.01

5. **Avg MSE: 2.4518, Std MSE: 0.4152**
   - Units1: 64, Activation1: relu, L2_reg: 0.001, Units2: 128, Units3: 32, Dropout Rate: 0.1, Learning Rate: 0.01

6. **Avg MSE: 2.5276, Std MSE: 0.4167**
   - Units1: 128, Activation1: relu, L2_reg: 0.01, Units2: 128, Units3: 128, Dropout Rate: 0.1, Learning Rate: 0.01

7. **Avg MSE: 2.6283, Std MSE: 0.5368**
   - Units1: 128, Activation1: relu, L2_reg: 0.001, Units2: 64, Units3: 128, Dropout Rate: 0.1, Learning Rate: 0.01

8. **Avg MSE: 2.6486, Std MSE: 0.5313**
   - Units1: 64, Activation1: relu, L2_reg: 0.01, Units2: 128, Units3: 32, Dropout Rate: 0.1, Learning Rate: 0.01

9. **Avg MSE: 2.6657, Std MSE: 0.6022**
   - Units1: 128, Activation1: relu, L2_reg: 0.001, Units2: 128, Units3: 32, Dropout Rate: 0.2, Learning Rate: 0.01

10. **Avg MSE: 2.7026, Std MSE: 0.4988**
    - Units1: 64, Activation1: relu, L2_reg: 0.01, Units2: 128, Units3: 64, Dropout Rate: 0.1, Learning Rate: 0.01


CPU times: user 40min 54s <br> sys: 2min 12s<br> total: 43min 6s<br>
Wall time: 42min 15s<br>
Total Combinations: 96



In [ ]:
%%time
param_grid = {
    'units1': [128],
    'units2': [128],
    'units3': [32, 128],  # let us consider variations around the best values
    'activation': ['relu'],  # Keeping 'relu' based its the best
    'learning_rate': [0.012, 0.014, 0.016, 0.018],  # Narrowing down around 0.001 with slight variations
    'dropout_rate': [0.012, 0.016, 0.1, 0.14],  # Expanding slightly around 0.1 to find a sweet spot
    'l2_reg': [0.015, 0.005, 0.01],  # Narrowing down around 0.01 with slight variations
}
top = perform_complete_grid_search(train_x, train_y, param_grid)

In [ ]:
print_model_collection(top)

MSE is from average of validation in folds

# Top 10 models architectures

**Model 1:**
- Average MSE: 1.3521, Std MSE: 0.2829
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 128
- activation3: relu
- dropout_rate: 0.012
- learning_rate: 0.014

**Model 2:**
- Average MSE: 1.3935, Std MSE: 0.2708
- units1: 128
- activation1: relu
- l2_reg: 0.01
- units2: 128
- activation2: relu
- units3: 32
- activation3: relu
- dropout_rate: 0.012
- learning_rate: 0.014

**Model 3:**
- Average MSE: 1.3967, Std MSE: 0.2706
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 32
- activation3: relu
- dropout_rate: 0.016
- learning_rate: 0.012

**Model 4:**
- Average MSE: 1.4285, Std MSE: 0.4004
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 128
- activation3: relu
- dropout_rate: 0.016
- learning_rate: 0.016

**Model 5:**
- Average MSE: 1.4374, Std MSE: 0.2155
- units1: 128
- activation1: relu
- l2_reg: 0.01
- units2: 128
- activation2: relu
- units3: 32
- activation3: relu
- dropout_rate: 0.012
- learning_rate: 0.016

**Model 6:**
- Average MSE: 1.4747, Std MSE: 0.1603
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 32
- activation3: relu
- dropout_rate: 0.012
- learning_rate: 0.014

**Model 7:**
- Average MSE: 1.4829, Std MSE: 0.3291
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 128
- activation3: relu
- dropout_rate: 0.012
- learning_rate: 0.012

**Model 8:**
- Average MSE: 1.4889, Std MSE: 0.3523
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 128
- activation3: relu
- dropout_rate: 0.016
- learning_rate: 0.012

**Model 9:**
- Average MSE: 1.4893, Std MSE: 0.0824
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 32
- activation3: relu
- dropout_rate: 0.016
- learning_rate: 0.014

**Model 10:**
- Average MSE: 1.4935, Std MSE: 0.1520
- units1: 128
- activation1: relu
- l2_reg: 0.005
- units2: 128
- activation2: relu
- units3: 32
- activation3: relu
- dropout_rate: 0.012
- learning_rate: 0.012


# GridSearchCV3

CPU times: user 42min 26s, sys: 2min 30s, total: 44min 57s
Wall time: 48min 28s
Combinations 54

In [ ]:
%%time

fixed_param_grid = {
    'units1': 128,
    'units2': 128,
    'units3': 128,
    'activation': 'relu',
    'l2_reg': 0.005,
    'dropout_rate': 0.012,
    'learning_rate': 0.014,

}

param_grid = {
    'batch_size': [16,32,64],  # try lower batches sizes than only 64
    'patience_es': [10,20],  # Exploring patience for early stopping
    'patience_lr': [5,10,15],  # Exploring patience for learning rate reduction
    'factor_lr': [0.05,0.1, 0.2]  # Exploring factors for learning rate reduction
}

# Assuming train_x and train_y are your training features and labels respectively
results = perform_callbacks_grid_search(train_x, train_y, param_grid, fixed_param_grid)

In [ ]:
print_model_collection(results)

MSE is from average of validation in folds

# Top Model Batches/Callbacks Configurations

* Model 1
- Average MSE: 1.0855
- Std MSE: 0.2459
- Batch Size: 16
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 10
- Learning Rate Reduction Factor: 0.2

## Model 2
- Average MSE: 1.1456
- Std MSE: 0.3840
- Batch Size: 16
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 10
- Learning Rate Reduction Factor: 0.1

## Model 3
- Average MSE: 1.1740
- Std MSE: 0.2918
- Batch Size: 16
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 15
- Learning Rate Reduction Factor: 0.1

## Model 4
- Average MSE: 1.2028
- Std MSE: 0.3623
- Batch Size: 64
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 15
- Learning Rate Reduction Factor: 0.2

## Model 5
- Average MSE: 1.2644
- Std MSE: 0.2890
- Batch Size: 16
- Patience for Early Stopping: 10
- Patience for Learning Rate Reduction: 5
- Learning Rate Reduction Factor: 0.2

## Model 6
- Average MSE: 1.2700
- Std MSE: 0.2878
- Batch Size: 16
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 5
- Learning Rate Reduction Factor: 0.2

## Model 7
- Average MSE: 1.3128
- Std MSE: 0.3890
- Batch Size: 64
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 15
- Learning Rate Reduction Factor: 0.1

## Model 8
- Average MSE: 1.3249
- Std MSE: 0.3849
- Batch Size: 32
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 10
- Learning Rate Reduction Factor: 0.2

## Model 9
- Average MSE: 1.3468
- Std MSE: 0.3799
- Batch Size: 32
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 15
- Learning Rate Reduction Factor: 0.1

## Model 10
- Average MSE: 1.3574
- Std MSE: 0.5765
- Batch Size: 32
- Patience for Early Stopping: 20
- Patience for Learning Rate Reduction: 10
- Learning Rate Reduction Factor: 0.05


# Matrics measure

CPU times: user 2min 21s<br> sys: 6.97 <br> total: 2min 28s <br>
Wall time: 2min 58s


In [ ]:
%%time
model_param_grid = {
    'units1': 128,
    'units2': 128,
    'units3': 128,
    'activation': 'relu',
    'l2_reg': 0.005,
    'dropout_rate': 0.012,
    'learning_rate': 0.014,

}

training_mse_scores = []
validation_mse_scores = []
training_mee_scores = []
validation_mee_scores = []
all_histories = []  # To store history objects from each fold

data_X = train_x
data_y = train_y
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

tf.keras.utils.set_random_seed(42)
for train_index, val_index in kf.split(data_X):
    X_train_fold, X_val_fold = data_X.iloc[train_index], data_X.iloc[val_index]
    y_train_fold, y_val_fold = data_y.iloc[train_index], data_y.iloc[val_index]
    scaler = StandardScaler()

    train_x_scaled = scaler.fit_transform(X_train_fold)
    val_scaled = scaler.transform(X_val_fold)#only transform, prevent leakage, use mean and std of training


    model = create_model(units1=model_param_grid['units1'], activation1=model_param_grid['activation'], l2_reg1=model_param_grid['l2_reg'],
                                             units2=model_param_grid['units2'], activation2=model_param_grid['activation'], l2_reg2=model_param_grid['l2_reg'],
                                             units3=model_param_grid['units3'], activation3=model_param_grid['activation'], l2_reg3=model_param_grid['l2_reg'],
                                             dropout_rate=model_param_grid['dropout_rate'], learning_rate=model_param_grid['learning_rate'])


    early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True)
    reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=0)
    additional_val_sets = {
        'second_val_set': (val_scaled, y_val_fold)
    }
    additional_validation_callback = AdditionalValidationSets(additional_val_sets, verbose=0)

    history = model.fit(train_x_scaled, y_train_fold,validation_split=0.1,
                        epochs=300, batch_size=16, verbose=1,callbacks=[early_stopping, reduce_lr_on_plateau,additional_validation_callback])
    train_pred = model.predict(train_x_scaled)
    val_pred = model.predict(val_scaled)

    train_mse = mean_squared_error(y_train_fold, train_pred)
    val_mse = mean_squared_error(y_val_fold, val_pred)
    train_mee = mean_euclidean_error(y_train_fold, train_pred)
    val_mee = mean_euclidean_error(y_val_fold, val_pred)


    training_mse_scores.append(train_mse)
    validation_mse_scores.append(val_mse)
    training_mee_scores.append(train_mee)
    validation_mee_scores.append(val_mee)
    all_histories.append(history)


avg_train_mse = np.mean(training_mse_scores)
std_train_mse = np.std(training_mse_scores)
avg_val_mse = np.mean(validation_mse_scores)
std_val_mse = np.std(validation_mse_scores)
avg_train_mee = np.mean(training_mee_scores)
std_train_mee = np.std(training_mee_scores)
avg_val_mee = np.mean(validation_mee_scores)
std_val_mee = np.std(validation_mee_scores)

print(f"Training average 5Fold MSE: {avg_train_mse:.4f} stdev: {std_train_mse:.4f}")
print(f"Validation average 5Fold MSE: {avg_val_mse:.4f} stdev: {std_val_mse:.4f}")
print(f"Training average 5Fold MEE: {avg_train_mee:.4f} stdev: {std_train_mee:.4f}")
print(f"Validation average 5Fold MEE: {avg_val_mee:.4f} stdev: {std_val_mee:.4f}")

- Training average 5Fold MSE: 0.4181 stdev: 0.0841
- Validation average 5Fold MSE: 1.2359 stdev: 0.3313
- Training average 5Fold MEE: 0.9011 stdev: 0.1063
- Validation average 5Fold MEE: 1.4002 stdev: 0.1537


In [ ]:
plot_aggregated_histories(all_histories)

In [ ]:
plot_aggregated_histories2(all_histories)

Final training.
We monitor loss on the training set and not the test set for EarlyStopping and ReduceLROnPlateau.


In [ ]:
%%time
model_param_grid = {
    'units1': 128,
    'units2': 128,
    'units3': 128,
    'activation': 'relu',
    'l2_reg': 0.005,
    'dropout_rate': 0.012,
    'learning_rate': 0.014,

}
tf.keras.utils.set_random_seed(42)

scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_scaled = scaler.transform(test_x)#only transform, prevent leakage, use mean and std of training
best_model = create_model(units1=model_param_grid['units1'], activation1=model_param_grid['activation'], l2_reg1=model_param_grid['l2_reg'],
                                             units2=model_param_grid['units2'], activation2=model_param_grid['activation'], l2_reg2=model_param_grid['l2_reg'],
                                             units3=model_param_grid['units3'], activation3=model_param_grid['activation'], l2_reg3=model_param_grid['l2_reg'],
                                             dropout_rate=model_param_grid['dropout_rate'], learning_rate=model_param_grid['learning_rate'])
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=0)


CPU times: user 78.9 ms, sys: 3 ms, total: 81.9 ms
Wall time: 78.7 ms


CPU times: user 19.8 s, sys: 1.28 s, total: 21.1 s
Wall time: 19.4 s


In [ ]:
%%time
tf.keras.utils.set_random_seed(42)
history = best_model.fit(train_x_scaled, train_y, validation_split = 0.1,
                            epochs=300, batch_size=16, callbacks=[early_stopping, reduce_lr_on_plateau], verbose=1)

Epoch 1/300
34/34 [==============================] - 1s 8ms/step - loss: 161.0970 - val_loss: 40.3823 - lr: 0.0140
Epoch 2/300
34/34 [==============================] - 0s 5ms/step - loss: 43.9605 - val_loss: 41.6479 - lr: 0.0140
Epoch 3/300
34/34 [==============================] - 0s 5ms/step - loss: 34.6807 - val_loss: 31.8799 - lr: 0.0140
Epoch 4/300
34/34 [==============================] - 0s 5ms/step - loss: 22.8530 - val_loss: 23.1789 - lr: 0.0140
Epoch 5/300
34/34 [==============================] - 0s 5ms/step - loss: 16.5511 - val_loss: 16.6560 - lr: 0.0140
Epoch 6/300
34/34 [==============================] - 0s 5ms/step - loss: 13.3243 - val_loss: 12.1130 - lr: 0.0140
Epoch 7/300
34/34 [==============================] - 0s 5ms/step - loss: 11.8896 - val_loss: 10.9483 - lr: 0.0140
Epoch 8/300
34/34 [==============================] - 0s 4ms/step - loss: 10.9948 - val_loss: 10.3704 - lr: 0.0140
Epoch 9/300
34/34 [==============================] - 0s 4ms/step - loss: 10.6405 - val_

In [ ]:
best_model.save("./bestmodelobtained")

INFO:tensorflow:Assets written to: ./bestmodelobtained/assets


INFO:tensorflow:Assets written to: ./bestmodelobtained/assets


In [ ]:
best_model = tf.keras.models.load_model('./bestmodelobtained')

In [ ]:
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)#only transform, prevent leakage, use mean and std of training

# Predict labels for the training set
train_labels_pred = best_model.predict(train_x_scaled)

# Predict labels for the test set
test_labels_pred = best_model.predict(test_x_scaled)

# Calculate MSE for the training set
train_mse = mean_squared_error(train_y, train_labels_pred)
print(f"Training MSE: {train_mse}")

# Calculate MSE for the test set
test_mse = mean_squared_error(test_y, test_labels_pred)
print(f"Test MSE: {test_mse}")

# Calculate MEE for the training set
train_mee = mean_euclidean_error(train_y, train_labels_pred)
print(f"Training MEE: {train_mee}")

# Calculate MEE for the test set
test_mee = mean_euclidean_error(test_y, test_labels_pred)
print(f"Test MEE: {test_mee}")


- Training MSE: 0.4488829589086582
- Test MSE: 1.5597875299329342
- Training MEE: 0.9347387567697988
- Test MEE: 1.7609950973231525


In [ ]:
best_model = tf.keras.models.load_model('./bestmodelobtained')

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(test_y, test_labels_pred)
plt.title("Best model of NN on Test Set - Predicted vs Actual")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")